In [9]:
from os import listdir
import pandas as pd
import numpy as np

# Converting the data to parquet for efficiency and not to use too much disk space

In [2]:
folder_in = "G:/dane_python/nyc_taxi/"
folder_out = "G:/dane_python/nyc_taxi_parquet/"

In [3]:
files_in = listdir(folder_in)

In [4]:
files_in

['yellow_tripdata_2020-01.csv',
 'yellow_tripdata_2020-02.csv',
 'yellow_tripdata_2020-03.csv',
 'yellow_tripdata_2020-04.csv',
 'yellow_tripdata_2020-05.csv',
 'yellow_tripdata_2020-06.csv',
 'yellow_tripdata_2020-07.csv',
 'yellow_tripdata_2020-08.csv',
 'yellow_tripdata_2020-09.csv',
 'yellow_tripdata_2020-10.csv',
 'yellow_tripdata_2020-11.csv',
 'yellow_tripdata_2020-12.csv',
 'yellow_tripdata_2021-01.csv',
 'yellow_tripdata_2021-02.csv',
 'yellow_tripdata_2021-03.csv',
 'yellow_tripdata_2021-04.csv',
 'yellow_tripdata_2021-05.csv',
 'yellow_tripdata_2021-06.csv',
 'yellow_tripdata_2021-07.csv']

In [5]:
data_types = {"VendorID": "Int64", 
              "tpep_pickup_datetime": "str",
             "tpep_dropoff_datetime": "str",
             "passenger_count": "Int64",
             "trip_distance": "float64",
             "RatecodeID": "Int64",
             "store_and_fwd_flag": "str",
             "PULocationID": "Int64",
             "DOLocationID": "Int64",
              "payment_type": "Int64",
              "fare_amount": "float64",
              "extra":"float64",
              "mta_tax":"float64",
              "tip_amount":"float64",
              "tolls_amount":"float64",
              "improvement_surcharge":"float64",
              "total_amount":"float64",
              "congestion_surcharge":"float64",
             }

date_cols = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]

In [6]:
geo_data_loc = "G:/dane_python/nyc_taxi_other/taxi_zones_lat_long.parquet"
geo_data = pd.read_parquet(geo_data_loc)

## Calculation of travel distance

Here I calculate haversine distance, based on two latitudes and longitudes (more here: https://en.wikipedia.org/wiki/Haversine_formula). Additionally, I calculate $L_1$ distance between pickup and dropoff points.

In [7]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
    a = haversine_array(lat1, lng1, lat1, lng2)
    b = haversine_array(lat1, lng1, lat2, lng1)
    return a + b

def bearing_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [10]:
for file in files_in:
    print('Reading file '+ file)
    df = pd.read_csv(folder_in + file,dtype = data_types,parse_dates = date_cols)
    
    print('Merging...')
    df1 = df.merge(geo_data,how='left',left_on="PULocationID",right_on="LocationID",suffixes=['','_PU'])\
        .merge(geo_data,how='left',left_on="DOLocationID",right_on="LocationID",suffixes=['','_DO'])
    df1 = df1.rename(columns={x : x + '_PU' for x in geo_data.columns})
    
    print('Calculating distances...')
    df1['distance_L2'] = haversine_array(df1['latitude_PU'].values, df1['longitude_PU'].values, df1['latitude_DO'].values, df1['longitude_DO'].values)
    df1['distance_L1'] = dummy_manhattan_distance(df1['latitude_PU'].values, df1['longitude_PU'].values, df1['latitude_DO'].values, df1['longitude_DO'].values)
    
    print('Converting file '+ file + ' to parquet')
    df1.to_parquet(folder_out + file.split('.')[0] + '.parquet', compression= None)

Reading file yellow_tripdata_2020-01.csv
Merging...
Calculating distances...
Converting file yellow_tripdata_2020-01.csv to parquet
Reading file yellow_tripdata_2020-02.csv
Merging...
Calculating distances...
Converting file yellow_tripdata_2020-02.csv to parquet
Reading file yellow_tripdata_2020-03.csv
Merging...
Calculating distances...
Converting file yellow_tripdata_2020-03.csv to parquet
Reading file yellow_tripdata_2020-04.csv
Merging...
Calculating distances...
Converting file yellow_tripdata_2020-04.csv to parquet
Reading file yellow_tripdata_2020-05.csv
Merging...
Calculating distances...
Converting file yellow_tripdata_2020-05.csv to parquet
Reading file yellow_tripdata_2020-06.csv
Merging...
Calculating distances...
Converting file yellow_tripdata_2020-06.csv to parquet
Reading file yellow_tripdata_2020-07.csv
Merging...
Calculating distances...
Converting file yellow_tripdata_2020-07.csv to parquet
Reading file yellow_tripdata_2020-08.csv
Merging...
Calculating distances...